In [1]:
import dataset_map_entry
import sys
import pickle
import os
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import csv



In [2]:
def load_dataset():
    input_dataset = open('./output/map_final.pkl', 'rb')
    dataset = pickle.load(input_dataset)
    input_dataset.close()

    return dataset

In [3]:
# custom spliter used instead of a tokenizer, since the tweets are already tokenized
def spaceSplitter(list):
    return list.split(" ")

def getFeatures(dataset, classification="both"):
    documents = []
    y = []

    # lemmatizer = WordNetLemmatizer()
    # stemmer = SnowballStemmer('english')

    # joining tokens with whitespace in order to fit the tf-idf vectorizer
    for user in sorted(dataset.keys()):
        tweets = dataset[user].get_tweets()
        if classification == "both":
            y.append(str(dataset[user].get_gender().value) + str(dataset[user].get_age_group().value))
        elif classification == "gender":
            y.append(str(dataset[user].get_gender().value))
        elif classification == "age":
            y.append(str(dataset[user].get_age_group().value))
        else:
            raise ValueError("Given clasification taks is not specified")

        document = []
        for tweet in tweets:
            # taggedTweet = nltk.pos_tag(tweet)
            # lemmatizedTweet = map( lambda x : lemmatizer.lemmatize(x[0], penn_to_wn(x[1])) , taggedTweet)
            # stemmedTweet = map( lambda x : stemmer.stem(x) , tweet)
            # document.append(" ".join(stemmedTweet))
            document.append(" ".join(tweet))
        documents.append(" ".join(document))

    ## Definining tf-idf vector

    vectorizer = TfidfVectorizer(tokenizer=spaceSplitter)
    vectorizer.fit(documents)

    features = vectorizer.transform(documents)
    names = vectorizer.get_feature_names()
    
    return features, y, names

In [4]:
ds = load_dataset()

In [5]:
features_age, y_age, names = getFeatures(dataset=ds, classification='age')


In [43]:
print(len(features_age.todense()))
print(len(y_age))
print(len(names))
i = 0
for name in names:
    column = names.index(name)
    print(len(features_age[:, column].todense()))
    features_age[:, column] = 0
    print(features_age[:, column])
    i += 1
    if i > 5:
        break

436
436
252169
436
  (0, 0)	0.0
  (1, 0)	0.0
  (2, 0)	0.0
  (3, 0)	0.0
  (4, 0)	0.0
  (5, 0)	0.0
  (6, 0)	0.0
  (7, 0)	0.0
  (8, 0)	0.0
  (9, 0)	0.0
  (10, 0)	0.0
  (11, 0)	0.0
  (12, 0)	0.0
  (13, 0)	0.0
  (14, 0)	0.0
  (15, 0)	0.0
  (16, 0)	0.0
  (17, 0)	0.0
  (18, 0)	0.0
  (19, 0)	0.0
  (20, 0)	0.0
  (21, 0)	0.0
  (22, 0)	0.0
  (23, 0)	0.0
  (24, 0)	0.0
  :	:
  (411, 0)	0.0
  (412, 0)	0.0
  (413, 0)	0.0
  (414, 0)	0.0
  (415, 0)	0.0
  (416, 0)	0.0
  (417, 0)	0.0
  (418, 0)	0.0
  (419, 0)	0.0
  (420, 0)	0.0
  (421, 0)	0.0
  (422, 0)	0.0
  (423, 0)	0.0
  (424, 0)	0.0
  (425, 0)	0.0
  (426, 0)	0.0
  (427, 0)	0.0
  (428, 0)	0.0
  (429, 0)	0.0
  (430, 0)	0.0
  (431, 0)	0.0
  (432, 0)	0.0
  (433, 0)	0.0
  (434, 0)	0.0
  (435, 0)	0.0
436
  (0, 0)	0.0
  (1, 0)	0.0
  (2, 0)	0.0
  (3, 0)	0.0
  (4, 0)	0.0
  (5, 0)	0.0
  (6, 0)	0.0
  (7, 0)	0.0
  (8, 0)	0.0
  (9, 0)	0.0
  (10, 0)	0.0
  (11, 0)	0.0
  (12, 0)	0.0
  (13, 0)	0.0
  (14, 0)	0.0
  (15, 0)	0.0
  (16, 0)	0.0
  (17, 0)	0.0
  (18, 0)	0.0


In [21]:
with open("./output/age_features.csv", "w") as file:
    writer = csv.writer(file, delimiter=";")
    writer.writerow(["UserId", "Phrase", "Score", "Class"])
    #writer.writerow(["Score", "Class"])
 
    doc_id = 0
    for doc, y in zip(features_age.todense(), y_age):
        #print("Document %d" %(doc_id))
        word_id = 0
        for score in doc.tolist()[0]:
            if score > 0:
                word = names[word_id].replace(";",",")
                writer.writerow([doc_id+1, word_id, score, y])
            word_id +=1
        doc_id +=1

In [62]:
df = pd.DataFrame()

for y in y_age:
    df = df.append([[features_age[y], y]], ignore_index=True)

df.columns = ['feature', 'class']


In [63]:
df.to_csv(path_or_buf='./output/age_features_tsv.tsv',sep='\t')